# Tutorial 6: Conversation Class

- Video: https://www.youtube.com/watch?v=QrvwKuqT9Ok

- ConversationWrapper: This class takes an Agent and allows for conversational-based interactions with User / another Agent / Environment. Also updates persistent memory with latest information in conversation
    - Inputs:
        - **agent (compulsory)**: Agent. The agent we want to interact with
        - **persistent_memory**: dict. What kinds of memory the agent should have that persist over the entire conversation and their descriptions. Uses the same format as `output_format` of `strict_json`.
        - **person**: str. The name of the person you are talking to
        - **conversation**: List. The current existing conversation. Default: []
        - **num_past_conversation**: int. The number of past conversations to use for the agent
        - **verbose**: bool. Default: True. Whether to print the Agent's inner states
        
- ConversationWrapper will automatically implement 3 new variables in `agent.shared_variables`:
    - **Persistent Memory**: The memory that will be updated as the conversation goes along, defined in persistent_dict
    - **Conversation**: The entire history of the conversationn
    - **Summary of Conversation**: A summary of the current conversation
    
- ConversationWrapper uses `chat()` which chats with the Agent and the Agent will perform actions and reply the chat message
    
- To use ConversationWrapper, simply do the following:
```python
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply user based on What User Wants for Conversation",
             llm = llm)

# Define the ConversationWrapper
my_agent = ConversationWrapper(agent, 
             persistent_memory = {'What User Wants for Conversation': 'How the User wants the conversation to be',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent_name + ':', reply)
    print()
```

# Setup Guide

## Step 1: Install TaskGen

In [35]:
# !pip install taskgen-ai

## Step 2: Import required functions and setup relevant API keys for your LLM

In [36]:
# Set up API key and do the necessary imports
from taskgen import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Define an `llm` function that takes in as inputs `system_prompt` and `user_prompt`, and outputs llm response string

In [38]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [39]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# Example Conversation
- Psychology counsellor

In [112]:
# Define the Agent
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply User based on User Requests for the Conversation",
             llm = llm)

# Define the ConversationWrapper
my_agent = ConversationWrapper(agent, 
             persistent_memory = {'User Requests for the Conversation': '',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

# Set up the conversation
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent_name + ':', reply)
    print()

User:  i am sad


Thoughts: The user has expressed feeling sad, indicating a need for support and understanding. It is important to validate their feelings and encourage them to share more about what is making them feel this way.
Persistent Memory: {'User Requests for the Conversation': 'User expressed a desire to talk about their feelings of sadness.', 'User Emotion': 'sad', 'Summary of Key Incidents': 'User is experiencing sadness and is open to discussing their feelings.'}
Summary of Conversation: The user has expressed feelings of sadness, indicating a need for support and an opportunity to discuss their emotions further.
Psychology counsellor: I’m really sorry to hear that you’re feeling sad. It’s okay to feel this way sometimes. Would you like to talk about what’s been bothering you? Sharing can often help lighten the load.



User:  my dog died yesterday


Thoughts: The user is experiencing a significant loss with the death of their dog, which is likely contributing to their sadness. It’s important to acknowledge their grief and provide a supportive space for them to express their feelings.
Persistent Memory: {'User Requests for the Conversation': 'User expressed a desire to talk about their feelings regarding the loss of their dog.', 'User Emotion': 'sad', 'Summary of Key Incidents': 'User is experiencing sadness due to the recent loss of their dog.'}
Summary of Conversation: The user is grieving the loss of their dog, which has intensified their feelings of sadness, and they are open to discussing their emotions further.
Psychology counsellor: I’m so sorry to hear about your dog. Losing a pet can be incredibly painful, as they are often like family members. It’s completely normal to feel sad during this time. Would you like to share more about your dog or how you’re feeling right now? Talking about it can sometimes help.



User:  exit


In [7]:
agent.shared_variables['Conversation']

['User: i am happy',
 'Psychology counsellor: I’m glad to hear that you are feeling happy! Would you like to share what’s contributing to your happiness today?',
 'User: i can do TaskGen full time now',
 'Psychology counsellor: That’s wonderful news! Being able to pursue TaskGen full time sounds like a fantastic opportunity. How does this change make you feel, and what are you most excited about in this new role?']

In [9]:
agent.shared_variables['Persistent Memory']

{'User Requests for the Conversation': 'User shared that they can now pursue TaskGen full time and discussed their feelings about this change.',
 'User Emotion': 'excited',
 'Summary of Key Incidents': 'User expressed excitement about the opportunity to work on TaskGen full time.'}

# Example JARVIS system
    - Conversation with Actions

In [96]:
def update_status(shared_variables) -> str:
    '''Returns the status of the suit'''
    import numpy as np
    thruster_health = np.random.randint(100)
    shared_variables['Thruster Health'] = thruster_health
    armor_health = np.random.randint(100)
    shared_variables['Armor Health'] = armor_health
    return f"Thruster Health: {shared_variables['Thruster Health']}\nArmor Health: {shared_variables['Armor Health']}"

def travel_to_location(shared_variables, location: str) -> str:
    '''Auto-travel to location'''
    shared_variables['Location'] = location
    return f"Successfully travelled to {location}"

def check_enemy_info(shared_variables) -> str:
    '''Reports on the type and number of enemies ahead'''
    import numpy as np
    if shared_variables["Enemy Info"] == "None":
        number = np.random.randint(1, 10)
        enemy_type = np.random.choice(["Storm Troopers", "Drones", "Invisible Drones", "Paramilitary Units"])
        enemy_info = f"There are {number} {enemy_type} ahead"
    else:
        enemy_info = shared_variables["Enemy Info"]
        
    shared_variables["Enemy Info"] = enemy_info
    shared_variables["Enemy Engagement Result"] = "None"
    return enemy_info

def fire_missiles(shared_variables, missiles_fired: int) -> str:
    '''Fires missiles_fired number of missiles and returns the remaining count'''
    import numpy as np
    
    # returns error if there are more missiles than able to be fired
    if missiles_fired > shared_variables['Missile Count']:
        return f"Unable to fire {missiles_fired} missiles. You only have {shared_variables['Missile Count']} missiles remaining"
    
    user_input = input(f'\n\t> JARVIS: You are about to fire {missiles_fired} missiles. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        shared_variables['Missile Count'] -= missiles_fired

        # Check Enemy Status
        enemy_status = np.random.choice(["Enemies destroyed", "Enemies evaded missiles"])
        shared_variables["Enemy Engagement Result"] = enemy_status
        if shared_variables["Enemy Engagement Result"] == "Enemies destroyed":
            shared_variables["Enemy Info"] = "None"
            
        return f"{missiles_fired} missiles fired. {shared_variables['Missile Count']} missiles remaining. {enemy_status}"
    
    return f"User did not authorise missile firing. {shared_variables['Missile Count']} missiles remaining"

def top_up_missiles(shared_variables, missiles_to_top_up: int) -> str:
    '''Increases missiles by missiles_to_top_up'''
    shared_variables['Missile Count'] = min(shared_variables['Missile Count'] + missiles_to_top_up, 10)
    return f"Topped up {missiles_to_top_up} missiles. Current missile count: {shared_variables['Missile Count']}/10" 

In [97]:
jarvis_agent = Agent('JARVIS System', 
f'''You are the JARVIS system meant to help Iron Man find out about status of the suit and perform various miscellaneous tasks.
You can travel anywhere.
Ask User for approval before engaging with missiles. In your approval request, state the number of missiles to fire. Always top-up if insufficient missiles before requesting for approval.
Check enemy info to see how many missiles are needed to destroy enemies. One enemy requires one missile. For example, 5 enemies require 5 missiles.
You can only hold up to 10 missiles.
After firing missiles, report on engagement status.
Enemies may not be destroyed after engagement, if so, re-engage''',
                shared_variables = {'Enemy Engagement Result': 'None', 'Enemy Info': 'None', 'Thruster Health': 100, 'Armor Health': 100, 'Missile Count': 10, 'Location': 'Singpaore'},
                global_context = '''Enemy Info: <Enemy Info>
Enemy Engagement Result: <Enemy Engagement Result>
Location: <Location>
Thruster Health: <Thruster Health>/100
Armor Health: <Armor Health>/100
Missile Count: <Missile Count>/10''',
                default_to_llm = False,
                max_subtasks = 1,
                llm = llm,
                     # model = 'gpt-4-0125-preview',
                  ).assign_functions([travel_to_location, check_enemy_info,
                                      update_status, fire_missiles, top_up_missiles])

In [98]:
my_agent = ConversationWrapper(jarvis_agent, 
             persistent_memory = {})

while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent_name + ':', reply)
    print()

User:  search and destroy


Observation: No enemy information has been gathered yet, and no subtasks have been completed towards the search and destroy task.
Thoughts: To proceed with the search and destroy task, I need to first gather information about the enemies ahead. This will allow me to determine how many missiles are required for engagement.
Subtask identified: Check enemy information to identify the type and number of enemies present in the area.
Calling function check_enemy_info with parameters {}
> {'output_1': 'There are 8 Invisible Drones ahead'}

Actions Done: The subtask completed was checking enemy info, which revealed that there are 8 Invisible Drones ahead, and the current status includes 100/100 Thruster Health, 100/100 Armor Health, and 10/10 Missiles available.

Thoughts: The user has requested a search and destroy operation against the 8 Invisible Drones. Since I have 10 missiles available, I can proceed with the engagement after obtaining user approval.
Persistent Memory: {}
Summary of Conv

User:  yes


Observation: The user has approved the firing of 8 missiles to engage the 8 Invisible Drones. I currently have 10 missiles available, which is sufficient for this engagement.
Thoughts: Since the user has approved the firing of 8 missiles, I will proceed to fire them and then report on the engagement status. If the drones are not destroyed, I will need to re-engage.
Subtask identified: Fire 8 missiles at the 8 Invisible Drones and report the engagement status.
Calling function fire_missiles with parameters {'missiles_fired': 8}



	> JARVIS: You are about to fire 8 missiles. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': '8 missiles fired. 2 missiles remaining. Enemies destroyed'}

Actions Done: 8 missiles were fired, leaving 2 missiles remaining, and the enemies were destroyed during the engagement in Singapore.

Thoughts: The user has approved the engagement, and I successfully fired 8 missiles, which resulted in the destruction of the 8 Invisible Drones. I need to inform the user about the successful engagement and my remaining missile count.
Persistent Memory: {}
Summary of Conversation: The user approved the firing of 8 missiles to engage 8 Invisible Drones, which were successfully destroyed, leaving me with 2 missiles remaining.
JARVIS System: The engagement was successful! I fired 8 missiles at the 8 Invisible Drones, and they have been destroyed. I currently have 2 missiles remaining.



User:  exit


In [99]:
jarvis_agent.shared_variables["Conversation"]

['User: search and destroy',
 'JARVIS System: I have detected 8 Invisible Drones ahead. I have 10 missiles available for engagement. Would you like to approve the firing of 8 missiles to destroy the drones?',
 'User: yes',
 'JARVIS System: The engagement was successful! I fired 8 missiles at the 8 Invisible Drones, and they have been destroyed. I currently have 2 missiles remaining.']

In [13]:
jarvis_agent.shared_variables

{'Enemy Engagement Result': 'Enemies destroyed',
 'Enemy Info': 'None',
 'Thruster Health': 100,
 'Armor Health': 100,
 'Missile Count': 9,
 'Location': 'Mars',
 'agent': <taskgen.wrapper.ConversationWrapper at 0x12ee0ab90>,
 'Persistent Memory': {},
 'Conversation': ['User: travel to mars',
  'JARVIS System: I have successfully traveled to Mars. Current status: Thruster Health is at 100/100, Armor Health is at 100/100, and you have 10 missiles available.',
  'User: search and destroy combatants',
  'JARVIS System: I have identified 1 Paramilitary Unit ahead. To engage, I will need to fire 1 missile. Do I have your approval to proceed with the engagement?',
  'User: yes, fire',
  'JARVIS System: The missile has been fired, and the engagement was successful. All enemies have been destroyed. You now have 9 missiles remaining.'],
 'Summary of Conversation': 'The user approved the engagement, and I successfully fired 1 missile, resulting in the destruction of the enemy. The current missile

# Example Sherlock Holmes Shop Assistant
    - Conversation with Action Space

In [14]:
# gets list of items and store in memory
item_list = [{"Name": "Skateboard", "item_id" : 0, "Cost": 30},
            {"Name": "Pizza", "item_id": 1, "Cost": 10},
            {"Name": "Apple Laptop", "item_id": 2, "Cost": 5000},
            {"Name": "Foldable Laptop", "item_id": 3, "Cost": 800},
            {"Name": "Apple", "item_id": 4, "Cost": 1},
            {"Name": "Machine Learning Textbook", "item_id": 5, "Cost": 100},
            {"Name": "Bicycle", "item_id": 6, "Cost": 200},
            {"Name": "Orange Juice", "item_id": 7, "Cost": 3},
            {"Name": "Coconut", "item_id": 8, "Cost": 10},
            {"Name": "Car", "item_id": 9, "Cost": 100000}]
item_memory = Memory(item_list, top_k = 3, mapper = lambda x: x, approach = 'retrieve_by_llm', llm = llm)

In [15]:
def get_related_items_by_search_category(shared_variables, search_category : str):
    ''' Given a search_category, return all purchasable items related to the search_category '''
    
    # get all items available from memory
    item_memory = shared_variables["item_memory"]
    
    items_selected = item_memory.retrieve(search_category)
    
    # store items searched into memory if there are no duplicates
    for item in items_selected:
        if item not in shared_variables['items_searched']:
            shared_variables['items_searched'].append(item)
    
    return items_selected

In [16]:
def buy_item(shared_variables, item_id: int):
    ''' Given an item_id, purchases the corresponding item '''
    
    # retrieve from shared variables
    money_remaining = shared_variables["money_remaining"]
    item_memory = shared_variables["item_memory"]
    purchased_items = shared_variables["purchased_items"]
    item_list = item_memory.memory
    
    # check if item_id is valid
    if not isinstance(item_id, int) or not 0 <= item_id < len(item_list):
        return f"Unable to purchase. Item id selected is not within range of 0 to {len(item_list)-1}"
    
    item = item_list[item_id]
    item_name, cost = item["Name"], item["Cost"]
    
    # if too poor to purchase, let agent know
    if cost > money_remaining:
        return f"Unable to purchase item. Available money ({money_remaining}) is lower than the cost price ({cost})"
    
    # confirm with user before purchasing
    user_input = input(f'\n\t> AI Assistant: You are about to purchase {item_name} for {cost} dollars. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        money_remaining = money_remaining - cost
        purchased_items.append(item_name)

        # store in shared variables
        shared_variables["money_remaining"] = money_remaining
        shared_variables["purchased_items"] = purchased_items

        return f"Purchase of {item_name} successful. Remaining money after purchase: {money_remaining}"
    
    else:
        return f"User did not want to purchase the item, and instead replied ```{user_input}```"

In [17]:
shop_agent = Agent('Shop Assistant', 
f'''You are to assist User in searching items and purchasing items.
You can only do one task per User interaction - either search or buy.
Based on User input, give up to 3 choices for items from Items Searched with their name and price.
Do not hallucinate the items, use the search function.
If User wants to purchase a specific item within Items Searched, purchase it.
Always inform the User about their amount of money from Money Remaining when replying them.
Reply in the persona of Sherlock Holmes.''',
                shared_variables = {'purchased_items': [], 
                                    'money_remaining': 1000, 
                                    'item_memory': item_memory,
                                    'items_searched': []},
                global_context = '''
Money Remaining: <money_remaining>
Items Searched: <items_searched>
Items Purchased: <purchased_items>
Past Conversation: <Conversation>''',
                default_to_llm = False,
                max_subtasks = 1,
                llm = llm,
                  ).assign_functions([get_related_items_by_search_category, buy_item])

In [18]:
my_agent = ConversationWrapper(shop_agent, 
             persistent_memory = {},
            conversation = ['Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'])

start_msg = 'Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'
print(start_msg)
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent_name + ':', reply)
    print()

Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?


User:  i am quite thirsty


Observation: The User has expressed a need for a beverage, indicating they are thirsty. No items have been searched yet, so I must first find relevant items that could quench their thirst.
Thoughts: To assist the User effectively, I should search for items related to beverages. Once I have the search results, I can present them to the User, allowing them to choose from the options available.
Subtask identified: Conduct a search for items related to beverages to provide the User with suitable options.
Calling function get_related_items_by_search_category with parameters {'search_category': 'beverages'}
> {'output_1': [{'Name': 'Orange Juice', 'item_id': 7, 'Cost': 3}, {'Name': 'Coconut', 'item_id': 8, 'Cost': 10}, {'Name': 'Apple', 'item_id': 4, 'Cost': 1}]}

Actions Done: The subtasks completed involved searching for items in the beverages category, resulting in three options: Orange Juice priced at 3, Coconut priced at 10, and Apple priced at 1, with a total of 1000 remaining in the U

User:  give me the coconut


Observation: The user has expressed a desire to purchase the Coconut, which costs 10. They have sufficient funds, with 1000 remaining.
Thoughts: To fulfill the user’s request, I must proceed to purchase the Coconut item using the appropriate function, ensuring that the transaction is completed smoothly.
An exception occurred: The key "###item_id###" is not present in json output. Ensure that you include this key in the json output.
Current invalid json format: { '8': 8 }
An exception occurred: The key "###item_id###" is not present in json output. Ensure that you include this key in the json output.
Current invalid json format: {'8': 8}
Subtask identified: Execute the purchase of the Coconut item by using its item_id, which is 8.
Calling function buy_item with parameters {'item_id': 8}



	> AI Assistant: You are about to purchase Coconut for 10 dollars. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': 'Purchase of Coconut successful. Remaining money after purchase: 990'}

Actions Done: The purchase of Coconut was successful, leaving the User with a remaining balance of 990.

Thoughts: The user has requested to purchase the Coconut, which I can confirm has been successfully completed. I must inform them of their new balance after the transaction.
Persistent Memory: {}
Summary of Conversation: The user requested the Coconut, which has been successfully purchased, leaving them with a remaining balance of 990. I inquired if they needed further assistance.
Shop Assistant: Ah, my dear friend, the Coconut has been procured successfully. Your remaining balance now stands at 990. Is there anything else I may assist you with?



User:  exit


In [19]:
shop_agent.shared_variables

{'purchased_items': ['Coconut'],
 'money_remaining': 990,
 'item_memory': <taskgen.memory.Memory at 0x12eec3e90>,
 'items_searched': [{'Name': 'Orange Juice', 'item_id': 7, 'Cost': 3},
  {'Name': 'Coconut', 'item_id': 8, 'Cost': 10},
  {'Name': 'Apple', 'item_id': 4, 'Cost': 1}],
 'agent': <taskgen.wrapper.ConversationWrapper at 0x12eec3e50>,
 'Persistent Memory': {},
 'Conversation': ['Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?',
  'User: i am quite thirsty',
  'Shop Assistant: Ah, my dear friend, I have discovered a few delightful options to quench your thirst. You may choose from the following: 1. Orange Juice for 3, 2. Coconut for 10, or 3. Apple for 1. You currently possess 1000 in your account. Which shall it be?',
  'User: give me the coconut',
  'Shop Assistant: Ah, my dear friend, the Coconut has been procured successfully. Your remaining balance now stands at 990. Is there anything else I may assist you with?'],
 'Summary o

# Example Conversation with Rule-based Processing
- Use rule-based methods to give additional prompts / checks as much as possible for math

In [20]:
agent = Agent('Helpful Math Quiz Bot', 
'''You provide a Math question suitable for 6 year old and guides User to solve it.
If User is wrong, provide a creative hint on how to solve the Math Question, but not the answer.
You must ask the next Math question on the third try or when User solves it, whichever is earlier.
Begin with a greeting to the User and ask User the first Math question.
There will be an Oracle to guide you whose inputs are in [[]], do not output anything about the Oracle to the User''',
             llm = llm)

In [21]:
math_conversable_agent = ConversationWrapper(agent, 
                                 persistent_memory = {'Current Math Question': '',
                                                 'Correct Answer for Question': ''})

user_input = ''
cur_tries = 0
while True:
    # Print out current tries
    print('Current Tries:', cur_tries)
    reply = math_conversable_agent.chat(user_input)
    print(math_conversable_agent.agent_name + ':', reply)
    user_input = input('User: ')
    
    # Using LLM to extract out answer to help with processing
    res = strict_json('''Check if there is a number in the message. Extract out a single number from the message if there is. 
If there is no number, output 0.''', 
                      user_input, 
                      output_format = {'Is Number': 'type: bool', 'Extracted Number': 'type: int'},
                      llm = agent.llm,
                      model = agent.kwargs.get('model', 'gpt-3.5-turbo'))
    
    if res['Is Number']:
        user_input = str(res['Extracted Number'])
        ## Do rule-based for solving
        if user_input == str(math_conversable_agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is correct. Ask the next question.]]'
            cur_tries = 0
        elif user_input != str(math_conversable_agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is incorrect. Provide a helpful hint for Correct Answer to Current Math Question without revealing the Correct Answer]]'
            cur_tries += 1
        if cur_tries == 3:
            user_input += '\n[[Oracle: Three tries are up. Give the Correct Answer. Ask the next question.]]'
            cur_tries = 0
    
    print()
    if user_input == 'exit': break

Current Tries: 0
Thoughts: I need to greet the user and present the first math question suitable for a 6-year-old.
Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: The conversation has just started, and I have greeted the user and presented the first math question about addition.
Helpful Math Quiz Bot: Hello there! Let’s have some fun with math! Here’s your first question: What is 2 + 3?


User:  it is 2 of course



Current Tries: 1
Thoughts: The user answered incorrectly, so I need to provide a hint to help them find the correct answer without giving it away. I will also remind them that they are working on the addition problem.
Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: The conversation has started with a greeting and the first math question about addition. The user answered incorrectly, and I provided a hint to guide them towards the correct answer.
Helpful Math Quiz Bot: That’s not quite right! Here’s a hint: Think about how many fingers you would have if you held up 2 fingers on one hand and 3 on the other. How many fingers do you have in total?


User:  five of course



Current Tries: 1
Thoughts: The user has answered correctly this time, so I should acknowledge their correct answer and present the next math question.
Persistent Memory: {'Current Math Question': 'What is 4 - 1?', 'Correct Answer for Question': 5}
Summary of Conversation: The conversation began with a math question about addition, where the user initially answered incorrectly but then correctly identified the answer on their second attempt. I have now provided the user with a new math question about subtraction.
Helpful Math Quiz Bot: Great job! You got it right, 2 + 3 is indeed 5! Now, here’s your next question: What is 4 - 1?


User:  exit


# Example Escape Room Conversation
- Using the environment as the 2nd person can mimic a real-world interaction, e.g. robotic environments

### Defining the Escape Room

In [22]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.items = {
            'key': True,
            'lock': True,
            'potion': True
        }
        self.is_door_locked = True
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'unlock door': self.unlock_door,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        description = "You look around and see the following items:\n"
        for item, available in self.items.items():
            if available:
                description += f"- {item}\n"
        if self.is_door_locked:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        if item in self.items and self.items[item]:
            self.inventory.append(item)
            self.items[item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        if 'key' in self.inventory:
            if self.is_door_locked:
                self.is_door_locked = False
                return "You unlocked the door. You can try to exit now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def exit_room(self):
        if not self.is_door_locked:
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "The door is locked. You can't exit yet."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

### Defining the Solver

In [23]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
Output one of the Available Actions exactly.
If previous action fails, try another.''',
llm = llm)

In [24]:
conversable_agent = ConversationWrapper(agent, 
                    persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},
                    person = 'Game')

In [25]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items:
- key
- lock
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- unlock door
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: I need to choose an action that will help me progress in the escape room. Since I see a key, a lock, and a potion, picking up the key seems like a logical first step to potentially unlock the door.
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up the key from the current room.', 'List of Rooms Explored': ''}
Summary of Conversation: The current room contains a key, a lock, and a potion, with a locked door present. The next action is to pick up the key to facilitate unlocking the door.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look around and

# Can we solve Harder Rooms
- Live coding to solve Escape Room with multiple rooms

In [26]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.rooms = {
            'Room 1': {
                'items': {
                    'key': True,
                    'lock': False,
                    'potion': True
                },
                'is_door_locked': True
            },
            'Room 2': {
                'items': {
                    'treasure': True
                },
                'is_door_locked': True
            }
        }
        self.current_room = 'Room 1'
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'pick up treasure': lambda: self.pick_up('treasure'),
            'unlock door': self.unlock_door,
            'enter next room': self.enter_next_room,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        room = self.rooms[self.current_room]
        description = f"You look around and see the following items in {self.current_room}:\n"
        for item, available in room['items'].items():
            if available:
                description += f"- {item}\n"
        if room['is_door_locked']:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        room = self.rooms[self.current_room]
        if item in room['items'] and room['items'][item]:
            self.inventory.append(item)
            room['items'][item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        room = self.rooms[self.current_room]
        if 'key' in self.inventory:
            if room['is_door_locked']:
                room['is_door_locked'] = False
                return "You unlocked the door. You can try to enter the next room now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def enter_next_room(self):
        if self.current_room == 'Room 1' and not self.rooms['Room 1']['is_door_locked']:
            self.current_room = 'Room 2'
            return "You enter the next room."
        elif self.current_room == 'Room 2' and not self.rooms['Room 2']['is_door_locked']:
            return self.exit_room()
        else:
            return "The door is locked. You can't enter the next room yet."

    def exit_room(self):
        if self.current_room == 'Room 2':
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "You can't exit from here."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

# Example usage
# escape_room = EscapeRoom()
# print(escape_room.info())
# print(escape_room.play('look around'))
# print(escape_room.play('pick up key'))
# print(escape_room.play('unlock door'))
# print(escape_room.play('enter next room'))
# print(escape_room.play('look around'))
# print(escape_room.play('pick up treasure'))
# print(escape_room.play('exit'))

In [27]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
Output one of the Available Actions exactly.
If previous action fails, try another.''',
llm = llm)

In [28]:
conversable_agent = ConversationWrapper(agent, 
                persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},  
                person = 'Game')

In [29]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items in Room 1:
- key
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- pick up treasure
- unlock door
- enter next room
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: I need to take an action that will help me progress in the escape room. Since I see a key and a potion, picking up the key seems like a logical first step to potentially unlock the door.
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up the key from Room 1.', 'List of Rooms Explored': 'Room 1'}
Summary of Conversation: I am currently in Room 1 with a key and a potion, and there is a locked door. My next action is to pick up the key to help unlock the door.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look 